In [ ]:
!sudo apt-get update &> /dev/null

In [ ]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet
!apt install openjdk-8-jdk-headless &> /dev/null

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('sem08').config('spark.ui.port', '4050').getOrCreate()

In [ ]:
spark

In [ ]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz &> /dev/null
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz &> /dev/null

In [ ]:
!./ngrok version

ngrok version 3.18.4


In [ ]:
!./ngrok authtoken token

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 5
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

https://6902-35-243-244-3.ngrok-free.app


---

In [ ]:
import pandas as pd

df = pd.DataFrame(
    data=[[1, 2], [2, 12], [3, 6], [4, 25]],
    columns=['a', 'b']
)
df

,a,b
0,1,2
1,2,12
2,3,6
3,4,25


In [ ]:
!rm -rf example_1
!mkdir example_1

df.to_parquet('/content/example_1/part-0000.parquet')

In [ ]:
!pwd

/content


In [ ]:
!rm -rf /content/example_1_output

from pyspark.sql import functions as f
from pyspark.sql.functions import col

spark.read.parquet('/content/example_1') \
     .withColumn('c', f.col('a') + 5) \
     .filter(f.col('b') > 10) \
     .write.parquet('/content/example_1_output')

In [ ]:
spark.read.parquet('/content/example_1') \
     .withColumn('c', f.col('a') + 5) \
     .filter(f.col('b') > 10) \
     .explain(True)

== Parsed Logical Plan ==
'Filter ('b > 10)
+- Project [a#12L, b#13L, (a#12L + cast(5 as bigint)) AS c#16L]
   +- Relation [a#12L,b#13L] parquet

== Analyzed Logical Plan ==
a: bigint, b: bigint, c: bigint
Filter (b#13L > cast(10 as bigint))
+- Project [a#12L, b#13L, (a#12L + cast(5 as bigint)) AS c#16L]
   +- Relation [a#12L,b#13L] parquet

== Optimized Logical Plan ==
Project [a#12L, b#13L, (a#12L + 5) AS c#16L]
+- Filter (isnotnull(b#13L) AND (b#13L > 10))
   +- Relation [a#12L,b#13L] parquet

== Physical Plan ==
*(1) Project [a#12L, b#13L, (a#12L + 5) AS c#16L]
+- *(1) Filter (isnotnull(b#13L) AND (b#13L > 10))
   +- *(1) ColumnarToRow
      +- FileScan parquet [a#12L,b#13L] Batched: true, DataFilters: [isnotnull(b#13L), (b#13L > 10)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/example_1], PartitionFilters: [], PushedFilters: [IsNotNull(b), GreaterThan(b,10)], ReadSchema: struct<a:bigint,b:bigint>



---

In [ ]:
!rm -rf /content/example_2_output_1
!rm -rf /content/example_2_output_2

df = spark.read.parquet('/content/example_1') \
          .withColumn('c', f.col('a') + 5)

df.filter(f.col('b') > 10).write.parquet('/content/example_2_output_1')
df.filter(f.col('b') <= 10).write.parquet('/content/example_2_output_2')

In [ ]:
!rm -rf /content/example_2_output_1
!rm -rf /content/example_2_output_2

df = spark.read.parquet('/content/example_1') \
          .withColumn('c', f.col('a') + 5) \
          .cache()

df.filter(f.col('b') > 10).write.parquet('/content/example_2_output_1')
df.filter(f.col('b') <= 10).write.parquet('/content/example_2_output_2')

---

In [ ]:
spark.read.parquet('/content/example_1')

DataFrame[a: bigint, b: bigint]

In [ ]:
spark.read.parquet('/content/example_1').explain(True)

== Parsed Logical Plan ==
Relation [a#196L,b#197L] parquet

== Analyzed Logical Plan ==
a: bigint, b: bigint
Relation [a#196L,b#197L] parquet

== Optimized Logical Plan ==
Relation [a#196L,b#197L] parquet

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [a#196L,b#197L] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/example_1], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<a:bigint,b:bigint>



In [ ]:
df = spark.read.parquet('/content/example_1')
df.select('a').explain(True)

== Parsed Logical Plan ==
'Project ['a]
+- Relation [a#171L,b#172L] parquet

== Analyzed Logical Plan ==
a: bigint
Project [a#171L]
+- Relation [a#171L,b#172L] parquet

== Optimized Logical Plan ==
Project [a#171L]
+- Relation [a#171L,b#172L] parquet

== Physical Plan ==
*(1) ColumnarToRow
+- FileScan parquet [a#171L] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/example_1], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<a:bigint>



In [ ]:
df = spark.read.parquet('/content/example_1')
df.select(f.count("*")).explain(True)

== Parsed Logical Plan ==
'Project [unresolvedalias(count(1), Some(org.apache.spark.sql.Column$$Lambda$3472/275390102@28ad1fc7))]
+- Relation [a#177L,b#178L] parquet

== Analyzed Logical Plan ==
count(1): bigint
Aggregate [count(1) AS count(1)#182L]
+- Relation [a#177L,b#178L] parquet

== Optimized Logical Plan ==
Aggregate [count(1) AS count(1)#182L]
+- Project
   +- Relation [a#177L,b#178L] parquet

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[count(1)], output=[count(1)#182L])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=134]
      +- HashAggregate(keys=[], functions=[partial_count(1)], output=[count#185L])
         +- FileScan parquet [] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/example_1], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<>



In [ ]:
!mkdir example_3
!mkdir example_3/a=1
!mkdir example_3/a=2

df = pd.DataFrame(
    data=[[1, 2], [1, 12], [1, 6], [1, 25]],
    columns=['a', 'b']
)
df.to_parquet('example_3/a=1/part1.parquet')

df = pd.DataFrame(
    data=[[2, 2], [2, 12], [2, 6], [2, 25]],
    columns=['a', 'b']
)
df.to_parquet('example_3/a=2/part1.parquet')

In [ ]:
df = spark.read.parquet('/content/example_3')
df.filter(f.col('a') == 2).explain(True)

== Parsed Logical Plan ==
'Filter ('a = 2)
+- Relation [a#200,b#201L] parquet

== Analyzed Logical Plan ==
a: int, b: bigint
Filter (a#200 = 2)
+- Relation [a#200,b#201L] parquet

== Optimized Logical Plan ==
Filter (isnotnull(a#200) AND (a#200 = 2))
+- Relation [a#200,b#201L] parquet

== Physical Plan ==
*(1) Project [a#200, b#201L]
+- *(1) ColumnarToRow
   +- FileScan parquet [b#201L,a#200] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/content/example_3], PartitionFilters: [isnotnull(a#200), (a#200 = 2)], PushedFilters: [], ReadSchema: struct<b:bigint>



---

In [ ]:
!rm -rf example_4
!mkdir example_4

import uuid
import random
import numpy as np

commands = ['read', 'write', 'other', None]
logins = [str(uuid.uuid4()) for _ in range(100)]

data = [[np.random.choice(commands, p=[0.5, 0.3, 0.15, 0.05]), random.choice(logins)] for _ in range(10000)]
df = pd.DataFrame(data=data, columns=['command', 'login'])
df.to_parquet('example_4/part-0000.parquet')
df.head(10)

,command,login
0,read,2fb172d3-8660-4ab1-84ba-1358ba34b792
1,write,5f78c475-0b32-4131-bba6-f3ad47ce36c1
2,read,18aeb910-275b-4583-b52e-c532baf83a3c
3,read,5ce23380-7790-4603-aa2b-6eefe48582af
4,read,addb1ed7-e82f-43ae-a051-2a3a9d7c7b6f
5,write,8b69cc1b-28de-4c05-827c-b7438e5374fc
6,read,c308e83e-3315-4a18-8139-17a25289f93b
7,write,5d7f233f-8dbd-461d-ac3d-dfa355c29175
8,write,4979c74b-d715-4f55-b454-3063142a91a1
9,None,b93327f3-dfd7-4a6d-9816-600b34358ce9


In [ ]:
df = spark.read.parquet('/content/example_4')

df.groupby('command') \
  .agg(
      f.count('*'),
      f.countDistinct('login')
  ) \
  .repartition(1) \
  .show()

+-------+--------+---------------------+
|command|count(1)|count(DISTINCT login)|
+-------+--------+---------------------+
|   NULL|     474|                  100|
|   read|    5067|                  100|
|  other|    1525|                  100|
|  write|    2934|                  100|
+-------+--------+---------------------+



In [ ]:
spark.conf.get('spark.sql.shuffle.partitions')

'200'

---

In [ ]:
!rm -rf example_5
!mkdir example_5

data = [[login, random.random() < 0.6] for login in logins]
df = pd.DataFrame(data=data, columns=['login', 'is_robot_flg'])
df.to_parquet('example_5/part-0000.parquet')
df

,login,is_robot_flg
0,c5e44745-325a-4269-897b-3443a5b014e6,False
1,a39fe21d-ee1f-4e11-9729-1f1cd6bf4387,True
2,55929ea9-5637-45cf-8345-0df1f2ba8826,True
3,1df1fe17-2823-4ba9-8fde-442787fdab67,False
4,c94f479d-a46c-492f-900c-363ca5917e7a,False
...,...,...
95,1fa4bc35-907a-4e81-b92d-aa1939b0e179,True
96,f7550dbd-c16f-4183-a104-4f008b0703ed,True
97,31348839-8764-4877-b439-7bc59ba2a662,True
98,9e86a765-63df-481c-bb17-481c3b4d5fca,True


In [ ]:
!rm -rf example_5_output
usage = spark.read.parquet('/content/example_4')
staff = spark.read.parquet('/content/example_5')

usage.join(staff, ['login'], 'inner') \
     .groupby('is_robot_flg') \
     .agg( f.count('*'), f.countDistinct('command') ) \
     .repartition(1) \
     .write.parquet('/content/example_5_output')

In [ ]:
usage = spark.read.parquet('/content/example_4')
staff = spark.read.parquet('/content/example_5')

usage.join(staff, ['login'], 'inner') \
     .groupby('is_robot_flg') \
     .agg( f.count('*'), f.countDistinct('command') ) \
     .repartition(1) \
     .show()

+------------+--------+-----------------------+
|is_robot_flg|count(1)|count(DISTINCT command)|
+------------+--------+-----------------------+
|        true|    5883|                      3|
|       false|    4117|                      3|
+------------+--------+-----------------------+



In [ ]:
from pyspark.sql.functions import broadcast

usage = spark.read.parquet('/content/example_4')
staff = spark.read.parquet('/content/example_5')

usage.join(broadcast(staff), ['login'], 'inner') \
     .groupby('is_robot_flg') \
     .agg( f.count('*'), f.countDistinct('command') ) \
     .repartition(1) \
     .show()

+------------+--------+-----------------------+
|is_robot_flg|count(1)|count(DISTINCT command)|
+------------+--------+-----------------------+
|        true|    5880|                      3|
|       false|    4120|                      3|
+------------+--------+-----------------------+



In [ ]:
# before
usage.join(broadcast(staff), ['login'], 'right_outer')

# after
tmp = usage.join(broadcast(staff), ['login'], 'inner')
staff.join(broadcast(tmp), ['login'], 'left_outer')

# right
keys = staff.select('login').distinct()
tmp = usage.join(broadcast(keys), ['login'], 'inner')
staff.join(broadcast(tmp), ['login'], 'left_outer')

DataFrame[login: string, is_robot_flg: boolean, command: string]